In [1]:
import gym
from stable_baselines3.ppo import CnnPolicy
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from pettingzoo.butterfly import cooperative_pong_v2
import supersuit as ss
import matplotlib
import numpy as np

from stable_baselines3.common.monitor import Monitor

from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common import results_plotter
from stable_baselines3.common.callbacks import BaseCallback

In [65]:
env = cooperative_pong_v2.parallel_env(ball_speed=9, left_paddle_speed=12, right_paddle_speed=12, cake_paddle=False, max_cycles=2000, bounce_randomness=False) #create env
agents= ['paddle_0', 'paddle_1'] #name agents
env = ss.color_reduction_v0(env, mode='B') #convert to grayscale for less computation
env = ss.resize_v0(env, x_size=84, y_size=84) #resive
env = ss.frame_stack_v1(env, 4)#stack 4 frames together to see velocity/direction
env = ss.pettingzoo_env_to_vec_env_v0(env) #convert to vec env
env = ss.concat_vec_envs_v0(env, 10, num_cpus=10, base_class='stable_baselines3') #parallelize

test_env = env

In [66]:
#model = PPO(CnnPolicy, env, verbose=3, gamma=0.99, n_steps=125, ent_coef=0.01, learning_rate=0.00025, vf_coef=0.5, max_grad_norm=0.5, gae_lambda=0.95, n_epochs=4, clip_range=0.2, clip_range_vf=1)
model = PPO(CnnPolicy, env,  learning_rate=0.0003, n_steps=2048, batch_size=64, n_epochs=10, gamma=0.99, gae_lambda=0.95, clip_range=0.2,
            clip_range_vf=1, ent_coef=0.0, vf_coef=0.5, max_grad_norm=0.5, use_sde=False, sde_sample_freq=- 1, target_kl=None, 
            tensorboard_log=None, create_eval_env=True, policy_kwargs=None, verbose=3, seed=314, device='auto', _init_setup_model=True)

model.learn(total_timesteps = (2000*100))
model.save("policy")

Using cpu device
Wrapping the env in a VecTransposeImage.
---------------------------------
| time/              |          |
|    fps             | 3260     |
|    iterations      | 1        |
|    time_elapsed    | 12       |
|    total_timesteps | 40960    |
| train/             |          |
|    learning_rate   | 0.0003   |
---------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 577         |
|    iterations           | 2           |
|    time_elapsed         | 141         |
|    total_timesteps      | 81920       |
| train/                  |             |
|    approx_kl            | -0.12568662 |
|    clip_fraction        | 0.433       |
|    clip_range           | 0.2         |
|    clip_range_vf        | 1           |
|    entropy_loss         | -1.07       |
|    explained_variance   | -2.43e-05   |
|    learning_rate        | 0.0003      |
|    loss                 | 55.4        |
|    n

In [74]:
#set up test env
env1 = cooperative_pong_v2.parallel_env(ball_speed=9, left_paddle_speed=12, right_paddle_speed=12, cake_paddle=False, max_cycles=2000, bounce_randomness=False) #create env
agents= ['paddle_0', 'paddle_1'] #name agents
env1 = ss.color_reduction_v0(env1, mode='B') #convert to grayscale for less computation
env1 = ss.resize_v0(env1, x_size=84, y_size=84) #resive
env1 = ss.frame_stack_v1(env1, 4)#stack 4 frames together to see velocity/direction
env1 = ss.pettingzoo_env_to_vec_env_v0(env1) #convert to vec env
env1 = ss.concat_vec_envs_v0(env1, 1, num_cpus=1, base_class='stable_baselines3') #parallelize, only 1 env

In [75]:
model1 = PPO.load("policy", env=env1)

Wrapping the env in a VecTransposeImage.


In [76]:

def evaluate(model, num_episodes=100):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    # This function will only work for a single Environment
    env = model.get_env()
    all_episode_rewards = []
    for i in range(num_episodes):
        episode_rewards = []
        done = False
        obs = env.reset()
        #while not done:
        # _states are only useful when using LSTM policies
        action, _states = model.predict(obs)
        # here, action, rewards and dones are arrays
        # because we are using vectorized env
        obs, reward, done, info = env.step(action)
        episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))

    mean_episode_reward = np.mean(all_episode_rewards)
    print("Mean reward:", mean_episode_reward, "Num episodes:", num_episodes)

    return mean_episode_reward

In [77]:
evaluate(model, num_episodes=1000)

Mean reward: 0.025000002 Num episodes: 1000


0.025000002